# labbox_ephys_widgets_jp example

**Hint: to avoid git conflicts, make a copy of this notebook prior to running. If changes are made, always clear the output cells prior to committing.**

In [ ]:
import spikeextractors as se
import numpy as np
import labbox_ephys as le
import labbox_ephys_widgets_jp as lew


In [ ]:
# Prepare recording and sorting extractors
# These must be of type le.LabboxEphysRecordingExtractor / le.LabboxEphysSortingExtractor
# However, the below code shows how to wrap any SpikeInterface extractors using from_memory(...)

def prepare_recording_sorting():
    # Simulate a recording (toy example)
    recording, sorting = se.example_datasets.toy_example(duration=50, num_channels=8, K=5, seed=1)
    R = le.LabboxEphysRecordingExtractor.from_memory(recording, serialize=True, serialize_dtype=np.int16)
    S = le.LabboxEphysSortingExtractor.from_memory(sorting, serialize=True)
    return R, S

recording, sorting = prepare_recording_sorting()

In [ ]:
# Launch the MVSortingView for visualization and to perform curation
# Try selecting one or more units and marking them as accepted

X = lew.sorting_views.MVSortingView(sorting=sorting, recording=recording)
display(X)

In [ ]:
# Now retrieve the curation object here
curation = X.get_curation()
display(curation)
# example output: {'labelsByUnit': {'1': ['accept'], '2': ['accept']}}

In [ ]:
# You can manipulate the curation via python and then set the mutated object back to the GUI
# Note: this doesn't seem to be working at the moment
if 'labelsByUnit' not in curation: curation['labelsByUnit'] = {}
curation['labelsByUnit']['4'] = ['reject']
X.set_curation(curation)

In [ ]:
# Similarly, retrieve the selected units
selection = X.get_selection()
display(selection)
# example output: {'selectedUnitIds': [1, 2, 5]}

In [ ]:
# You can set external metrics that will appear in the units table
m = [{'name': 'test', 'label': 'Test metric', 'tooltip': 'Test metric', 'data': {'1': 10, '2': 20, '3': 30}}]
X.set_external_unit_metrics(m)

In [ ]:
# It is also possible to view individual widgets (instead of launching MVSortingView)
lew.sorting_views.AverageWaveforms(sorting=sorting, recording=recording)

In [ ]:
lew.sorting_views.ElectrodeGeometry(sorting=sorting, recording=recording)

In [ ]:
lew.sorting_views.Autocorrelograms(sorting=sorting, recording=recording)

In [ ]:
lew.sorting_views.CrossCorrelograms(sorting=sorting, recording=recording)

In [ ]:
lew.sorting_views.Timeseries(recording=recording, sorting=sorting)